In [1]:
%matplotlib inline
import quandl
import seaborn as sns
import pandas as pd
token='QWe8iSbyAFzRuod2aroM'

/Users/mrefermat/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
/Users/mrefermat/anaconda/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated since IPython 4.0.You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [ ]:
import pylab
sns.set(font_scale=3)
pylab.rcParams['figure.figsize'] = (24, 18)

In [ ]:
mkts={'SP 500':'CHRIS/CME_SP1',
      'US 10Y':'CHRIS/CME_TY1',
      'Crude Oil':'CHRIS/CME_CL1',
      'Eurostoxx 50':'CHRIS/EUREX_FESX1'
      }

In [ ]:
data_index=pd.DataFrame()
for m in mkts.keys():
    try:
        data_index[m]=quandl.get(mkts[m],token=token).Last
    except:
        data_index[m]=quandl.get(mkts[m],token=token).Settle
data_pct=data_index.pct_change()     

In [ ]:
data_pct['2018':].cumsum().ffill().plot(colormap='brg').get_figure().savefig('test.png')

In [ ]:
data_pct['2018':].cumsum().ffill().plot(colormap='brg')

In [1]:
import boto3
from botocore.exceptions import ClientError

In [3]:

SENDER = "mark.refermat@gmail.com"
RECIPIENT = "mrefermat@hotmail.com"

# If necessary, replace us-west-2 with the AWS Region you're using for Amazon SES.
AWS_REGION = "us-east-1"

# The subject line for the email.
SUBJECT = "Amazon SES Test (SDK for Python) from my computer"

# The email body for recipients with non-HTML email clients.
BODY_TEXT = ("Amazon SES Test (Python)\r\n"
             "This email was sent with Amazon SES using the "
             "AWS SDK for Python (Boto)."
            )
            
# The HTML body of the email.
BODY_HTML = """<html>
<head></head>
<body>
  <h1>Amazon SES Test (SDK for Python)</h1>
  <p>This email was sent with
    <a href='https://aws.amazon.com/ses/'>Amazon SES</a> using the
    <a href='https://aws.amazon.com/sdk-for-python/'>
      AWS SDK for Python (Boto)</a>.</p>
</body>
</html>
            """            

# The character encoding for the email.
CHARSET = "UTF-8"

# Create a new SES resource and specify a region.
client = boto3.client('ses',region_name=AWS_REGION)

# Try to send the email.
try:
    #Provide the contents of the email.
    response = client.send_email(
        Destination={
            'ToAddresses': [
                RECIPIENT,
            ],
        },
        Message={
            'Body': {
                'Html': {
                    'Charset': CHARSET,
                    'Data': BODY_HTML,
                },
                'Text': {
                    'Charset': CHARSET,
                    'Data': BODY_TEXT,
                },
            },
            'Subject': {
                'Charset': CHARSET,
                'Data': SUBJECT,
            },
        },
        Source=SENDER,
    )
# Display an error if something goes wrong.     
except ClientError as e:
    print(e.response['Error']['Message'])
else:
    print("Email sent! Message ID:"),
    print(response['MessageId'])





Email sent! Message ID:
0100016396ac97fc-0ef24414-748a-46c1-80c7-76f0905fad73-000000


In [46]:
connection = boto3.client('ses',region_name='us-east-1')

In [47]:
connection.send_raw_email?

Signature: connection.send_raw_email(*args, **kwargs)
Docstring:
Composes an email message and immediately queues it for sending. When calling this operation, you may specify the message headers as well as the content. The ``SendRawEmail`` operation is particularly useful for sending multipart MIME emails (such as those that contain both a plain-text and an HTML version). 

 

In order to send email using the ``SendRawEmail`` operation, your message must meet the following requirements:

 

 
* The message must be sent from a verified email address or domain. If you attempt to send email using a non-verified address or domain, the operation will result in an "Email address not verified" error.  
 
* If your account is still in the Amazon SES sandbox, you may only send to verified addresses or domains, or to email addresses associated with the Amazon SES Mailbox Simulator. For more information, see `Verifying Email Addresses and Domains <http://docs.aws.amazon.com/ses/latest/DeveloperGu

In [126]:
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage

import boto3

class Email(object):

    def __init__(self, to, subject):
        self.to = to
        self.subject = subject
        self.text = None
        self.attachment = None
        self.msg = MIMEMultipart()
        
    def add_text(self, text):
        self.text = text

    def add_attachment(self, file_name):
        msgText = MIMEText('<br><img src="cid:%s"><br>' %  file_name, 'html')  
        self.msg.attach(msgText) 
        fp=open(file_name,'rb')
        self.attachment = MIMEImage(fp.read())
        fp.close()
        self.attachment.add_header('Content-ID', '<{}>'.format(file_name))
        self.msg.attach(self.attachment)
        
    def add_attachments(self, list_of_files):
        m=''
        for file_name in list_of_files:
            m=m+'<br><img src="cid:%s"><br>' %  file_name       
        msgText = MIMEText(m, 'html')  
        self.msg.attach(msgText) 
        for file_name in list_of_files:    
            fp=open(file_name,'rb')
            self.attachment = MIMEImage(fp.read())
            fp.close()
            self.attachment.add_header('Content-ID', '<{}>'.format(file_name))
            self.msg.attach(self.attachment)
        
    def send(self):
        from_addr='mrefermat@hotmail.com'
        connection = boto3.client('ses',region_name="us-east-1")
        self.msg['Subject'] = self.subject
        self.msg['From'] = from_addr
        self.msg['To'] = self.to

        return connection.send_raw_email(RawMessage={'Data': self.msg.as_string()},
                                         Source=from_addr,
                                         Destinations=[self.to])


In [127]:
e=Email(to='mark.refermat@gmail.com',subject='Two pictures')

In [128]:
e.add_attachments(['test.png','test.png'])

In [129]:
e.send()

{'MessageId': '0100016397045dfd-698c407a-bcb8-4c8e-b144-63be36498fc6-000000',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '338',
   'content-type': 'text/xml',
   'date': 'Fri, 25 May 2018 11:17:15 GMT',
   'x-amzn-requestid': '2cc66144-600d-11e8-b840-595e854ad328'},
  'HTTPStatusCode': 200,
  'RequestId': '2cc66144-600d-11e8-b840-595e854ad328',
  'RetryAttempts': 0}}